In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
import os
import bokeh
from tqdm import tqdm

In [2]:
version_name = '1024base.1025fix.dl'
version_path_str = f'/workspace/code/{version_name}'

In [3]:
ckpts = os.popen(f'ls {version_path_str}/ckpt', 'r')
results = dict()
for ckpt in ckpts.readlines():
    epoch, dev_loss = ckpt.strip().split('-') 
    epoch = int(epoch)
    dev_loss = float(dev_loss)
    results[epoch] = dev_loss
    
dev_list = []
for i in range(1, len(results) + 1):
    dev_list.append(results[i])
print(dev_list)

[4.303, 3.9867, 3.8774, 3.7879, 3.7281, 3.7098, 3.6991, 3.7194, 3.7388, 3.763, 3.8016, 3.8469, 3.8881, 3.9337, 3.9608, 4.0199, 4.0723, 4.096, 4.1745, 4.2156, 4.2606, 4.3033, 4.3531, 4.4063, 4.4415, 4.4898, 4.5329, 4.5773, 4.5872, 4.6461]


In [4]:
results = os.popen(f'ls {version_path_str}/result', 'r')
results = filter(str.isdigit, results.read().strip().split('\n'))
res_list = [int(r) for r in results]
res_list.sort()

In [5]:
bleu_results = []
for r in tqdm(res_list):
    with open(f'{version_path_str}/result/{r}', encoding='utf-8') as f:
        lines = f.readlines()
    tgts = []
    for i, line in enumerate(lines):
        if i % 2 == 1:
            tgts.append(line)
    with open(f'{version_path_str}/result/{r}-tgt', 'w', encoding='utf-8') as f:
        f.writelines(tgts)
    bleu_r = os.popen(f'perl ../../multi-bleu.perl ../../raw/ijcnlp_dailydialog/test_tgt.txt < {version_path_str}/result/{r}-tgt', 'r')
    bleu_r4score = [float(score) for score in bleu_r.read().split(' ')[3].strip().split('/')]
    bleu_results.append(bleu_r4score)

bleu_4result = list(zip(*bleu_results))
print(bleu_4result)

100%|██████████| 30/30 [01:18<00:00,  2.47s/it]

[(9.0, 22.2, 21.9, 24.4, 25.3, 21.2, 23.2, 25.5, 21.8, 24.6, 21.9, 23.4, 24.7, 24.8, 25.5, 24.5, 25.3, 25.9, 25.6, 26.7, 25.6, 26.1, 27.2, 27.0, 27.7, 28.6, 28.3, 29.1, 29.5, 29.7), (1.0, 2.7, 3.1, 3.7, 4.0, 3.6, 3.9, 4.2, 3.9, 4.4, 4.1, 4.6, 4.9, 5.3, 5.8, 5.7, 6.3, 6.9, 7.1, 8.0, 7.5, 8.3, 9.2, 9.3, 10.1, 10.7, 11.2, 11.9, 12.3, 12.7), (0.2, 0.7, 0.8, 1.2, 1.5, 1.1, 1.3, 1.5, 1.4, 1.7, 1.5, 1.9, 2.3, 2.5, 2.8, 2.9, 3.6, 4.1, 4.3, 5.2, 4.8, 5.6, 6.4, 6.7, 7.4, 8.1, 8.5, 9.2, 9.7, 10.2), (0.1, 0.2, 0.2, 0.4, 0.5, 0.4, 0.4, 0.6, 0.5, 0.8, 0.7, 1.0, 1.3, 1.5, 1.8, 2.0, 2.5, 3.0, 3.2, 4.1, 3.7, 4.6, 5.3, 5.6, 6.2, 7.0, 7.4, 8.1, 8.7, 9.2)]


In [6]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import LinearAxis, Range1d

fig = figure(title="Trend", width=1000, x_axis_label='epoch', y_axis_label='value')
fig.extra_y_ranges = {"loss": Range1d(start=2.5, end=5)}
fig.add_layout(LinearAxis(y_range_name="loss"), 'right')
fig.grid.grid_line_alpha=0.3
fig.xaxis.axis_label = 'epoch'
fig.yaxis.axis_label = 'value'

xaxis = list(range(1, len(dev_list) + 1))
fig.quad(left=[0] + xaxis[:-1], right=xaxis, top=dev_list, bottom=0,
         fill_color="navy", line_color="white", alpha=0.5, legend='dev loss', y_range_name="loss")
fig.line(xaxis, bleu_4result[0], color="blue", line_width=4, alpha=0.7, legend='bleu1')
fig.line(xaxis, bleu_4result[1], color="green", line_width=4, alpha=0.7, legend='bleu2')
fig.line(xaxis, bleu_4result[2], color="orange", line_width=4, alpha=0.7,legend='bleu3')
fig.line(xaxis, bleu_4result[3], color="red", line_width=4, alpha=0.7,   legend='bleu4')
fig.legend.location = "top_left"

output_file(f"{version_name}.html", title="Trend")

show(fig)  # open a browser